In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))


def split_train_test(df, test_size):
    df = df.sort_values('ds').reset_index(drop=True)
    train_size = int(len(df) * (1 - test_size))
    return df.iloc[:train_size], df.iloc[train_size:]